In [6]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind
import numpy as np

In [2]:
# Load your CSV or paste raw data
df = pd.read_csv("/Users/jon/Desktop/Ironhack/Unit 4 - Statistics & Probability/Project-Statistics/Databases/cleaned/df_cleaned_web_data_vfinal.txt", parse_dates=["date_time"])

CALCULATION OF AVERAGE TIMINGS BY STEP

In [3]:
# Group by 'process_step' and calculate the average time
avg_time_per_step = df.groupby("process_step")["time_spent"].mean().reset_index()

# Sort by a logical step order if needed
step_order = ["start", "step_1", "step_2", "step_3", "confirm"]
avg_time_per_step["step_order"] = avg_time_per_step["process_step"].apply(lambda x: step_order.index(x) if x in step_order else -1)
avg_time_per_step = avg_time_per_step.sort_values("step_order").drop(columns="step_order")

# Display result
print(avg_time_per_step)

  process_step  time_spent
0        start   58.696367
1       step_1   60.524441
2       step_2   92.450280
3       step_3  132.116471


FUNNEL OF CONVERSION

In [5]:
# Sort actions chronologically per visit
df = df.sort_values(by=["visit_id", "date_time"])

# Remove duplicate steps per visit (to avoid loops/skips being counted multiple times)
df_unique_steps = df.drop_duplicates(subset=["visit_id", "process_step"])

# Define the correct step order for the funnel
step_order = ["start", "step_1", "step_2", "step_3"]

# Count how many unique visits reached each step
step_counts = df_unique_steps.groupby("process_step")["visit_id"].nunique().reindex(step_order, fill_value=0)

# Calculate funnel conversion from the first step
funnel = step_counts.to_frame(name="num_visits")
funnel["conversion_rate"] = funnel["num_visits"] / funnel["num_visits"].iloc[0] * 100  # % from start

# Display results
print("Funnel Conversion (% of users from 'start'):\n")
print(funnel)

Funnel Conversion (% of users from 'start'):

              num_visits  conversion_rate
process_step                             
start             130490       100.000000
step_1            110012        84.306843
step_2             99790        76.473293
step_3             88738        68.003678


Avg Timing per Step, Fastest Group, and Significance

In [7]:
#Merge both databases i'll be using

# Load the web steps data with date_time parsed as datetime
df_web = pd.read_csv(
    "/Users/jon/Desktop/Ironhack/Unit 4 - Statistics & Probability/Project-Statistics/Databases/cleaned/df_cleaned_web_data_vfinal.txt",
    parse_dates=["date_time"]
)

# Load the experiment clients data
df_exp = pd.read_csv(
    "/Users/jon/Desktop/Ironhack/Unit 4 - Statistics & Probability/Project-Statistics/Databases/cleaned/df_cleaned_experiment_clients_data.txt"
)

In [8]:
# Merge experiment variation info into web steps data
df_merged = df_web.merge(df_exp, on="client_id", how="inner")

# Sort by visit_id and date_time to ensure correct order
df_merged = df_merged.sort_values(by=["visit_id", "date_time"])

# Calculate next timestamp per visit (for duration calculation)
df_merged["next_time"] = df_merged.groupby("visit_id")["date_time"].shift(-1)

# Calculate time spent in seconds on each step
df_merged["time_spent"] = (df_merged["next_time"] - df_merged["date_time"]).dt.total_seconds()

# Remove rows where next_time is null (last step in visit)
df_valid = df_merged.dropna(subset=["time_spent"]).copy()

# Optional: show first rows for verification
print(df_valid.head())

   client_id             visitor_id                     visit_id process_step  \
0    7338123  612065484_94198474375  100019538_17884295066_43909        start   
1    7338123  612065484_94198474375  100019538_17884295066_43909       step_1   
2    7338123  612065484_94198474375  100019538_17884295066_43909       step_2   
3    7338123  612065484_94198474375  100019538_17884295066_43909       step_1   
4    7338123  612065484_94198474375  100019538_17884295066_43909       step_1   

            date_time           next_time  time_spent Variation  
0 2017-04-09 16:20:56 2017-04-09 16:21:12        16.0      Test  
1 2017-04-09 16:21:12 2017-04-09 16:21:21         9.0      Test  
2 2017-04-09 16:21:21 2017-04-09 16:21:35        14.0      Test  
3 2017-04-09 16:21:35 2017-04-09 16:21:41         6.0      Test  
4 2017-04-09 16:21:41 2017-04-09 16:21:45         4.0      Test  


Step 2: Average Timings + T-tests per Step & Variation

In [9]:
# Group by step and variation to calculate mean and count
grouped = df_valid.groupby(["process_step", "Variation"])["time_spent"]
avg_time = grouped.mean().unstack()       # avg time per step per group
counts = grouped.count().unstack()        # counts per step per group

# List of unique process steps
steps = df_valid["process_step"].unique()

# Prepare results list
results = []

for step in steps:
    # Extract time_spent for Test and Control groups for this step
    test_times = df_valid[(df_valid["process_step"] == step) & (df_valid["Variation"] == "Test")]["time_spent"]
    control_times = df_valid[(df_valid["process_step"] == step) & (df_valid["Variation"] == "Control")]["time_spent"]
    
    # Perform t-test if we have enough data (>1 in each group)
    if len(test_times) > 1 and len(control_times) > 1:
        stat, pval = ttest_ind(test_times, control_times, equal_var=False)  # Welch's t-test
    else:
        pval = np.nan  # Not enough data
    
    # Identify faster group
    faster_group = None
    if step in avg_time.index:
        if avg_time.loc[step, "Test"] < avg_time.loc[step, "Control"]:
            faster_group = "Test"
        elif avg_time.loc[step, "Control"] < avg_time.loc[step, "Test"]:
            faster_group = "Control"
    
    results.append({
        "process_step": step,
        "avg_time_test": avg_time.loc[step, "Test"] if "Test" in avg_time.columns else np.nan,
        "avg_time_control": avg_time.loc[step, "Control"] if "Control" in avg_time.columns else np.nan,
        "test_n": counts.loc[step, "Test"] if "Test" in counts.columns else 0,
        "control_n": counts.loc[step, "Control"] if "Control" in counts.columns else 0,
        "faster_group": faster_group,
        "p_value": pval
    })

# Convert results list to DataFrame
results_df = pd.DataFrame(results)

# Optional: sort steps in natural order if you want
step_order = ['start', 'step_1', 'step_2', 'step_3', 'confirm']
results_df["step_order"] = results_df["process_step"].apply(lambda x: step_order.index(x) if x in step_order else 99)
results_df = results_df.sort_values("step_order").drop(columns="step_order")
results_df["significant"] = results_df["p_value"].apply(lambda p: "Yes" if p < 0.05 else "No")

# Show results
print(results_df)

  process_step  avg_time_test  avg_time_control  test_n  control_n  \
0        start      58.438816         65.903188   41359      31308   
1       step_1      61.021054         50.635360   32631      23903   
2       step_2      87.922946         91.502835   27046      22047   
3       step_3     222.037115        176.775640    4769       5197   

  faster_group       p_value significant  
0         Test  1.958198e-04         Yes  
1      Control  7.395664e-16         Yes  
2         Test  1.684758e-02         Yes  
3      Control  2.659241e-09         Yes  


Step 2: Total Average Timings + T-tests  & Variation

In [10]:
# Sum time_spent per visit and variation
total_time_per_visit = df_valid.groupby(["visit_id", "Variation"])["time_spent"].sum().reset_index()

# Average total time per group
avg_total_time = total_time_per_visit.groupby("Variation")["time_spent"].mean()

# Counts per group (number of visits)
counts = total_time_per_visit["Variation"].value_counts()

# Extract total times per group for t-test
test_times = total_time_per_visit[total_time_per_visit["Variation"] == "Test"]["time_spent"]
control_times = total_time_per_visit[total_time_per_visit["Variation"] == "Control"]["time_spent"]

# Run Welch's t-test
stat, p_value = ttest_ind(test_times, control_times, equal_var=False)

# Determine faster group
if avg_total_time.get("Test", np.nan) < avg_total_time.get("Control", np.nan):
    faster_group = "Test"
else:
    faster_group = "Control"

# Create a summary DataFrame with the same style as your example
summary_total_time = pd.DataFrame({
    "process_step": ["total_time"],
    "avg_time_test": [avg_total_time.get("Test", np.nan)],
    "avg_time_control": [avg_total_time.get("Control", np.nan)],
    "test_n": [counts.get("Test", 0)],
    "control_n": [counts.get("Control", 0)],
    "faster_group": [faster_group],
    "p_value": [p_value],
    "significant": ["Yes" if p_value < 0.05 else "No"]
})

print(summary_total_time)

  process_step  avg_time_test  avg_time_control  test_n  control_n  \
0   total_time     285.314518        276.650628   27496      22446   

  faster_group   p_value significant  
0      Control  0.074391          No  
